# HP/Agilent/Keysight 6060B System DC Electronic Load calibration

Use this script as a starting point only. It might contain bugs.

## User and calibration point settings

### Connection settings

In [30]:
load_res_id = "GPIB0::12::INSTR"

### Calibration settings and points

When cal_save is off, the calibration values are not automatically stored and will be lost if power is cycled before sending `CAL:SAVE`

In [31]:
cal_save = True

The modifier scales certain values up or down (default = 1). This can be used to calibrate the unit even if the power supplies/current meters used do not cover the range of required settings fully. Before changing understand the code. Be careful as this might DEGRADE the performance of the unit and IS NOT how HP specified the calibration/adjustment procedure.

In [32]:
cal_set_curr_rng_hi = { "current_range": 60, "current_offset": 0.0282, "power_supply": { "voltage": 5, "current": 61}, "flag": True, "modifier": 1}
cal_set_curr_rng_lo = { "current_range": 6, "current_offset": 0.0197, "power_supply": { "voltage": 5, "current": 10}, "flag": False, "modifier": 1}

cal_set_volt = { "voltage_high": 60, "voltage_low": 2.7, "power_supply": {"voltage": 61, "current": 5}}

cal_set_res_rng_lo = { "res_range": 1, "res_low": 0.04, "res_high": 1, "power_supply": {"voltage": 15, "current": 10.9}, "flag": False, "modifier": 1}
cal_set_res_rng_med = { "res_range": 10, "res_low": 1, "res_high": 30, "power_supply": {"voltage": 10.9, "current": 15}, "flag": True, "modifier": 1}
cal_set_res_rng_hi = { "res_range": 1001, "res_low": 12, "res_high": 120, "power_supply": {"voltage": 60, "current": 6}, "flag": True, "modifier": 1}

## Imports

In [33]:
import sys
sys.path.insert(0, ".")

import pyvisa
import numpy as np
import pandas as pd
from time import sleep
from datetime import datetime
from pathlib import Path

from scpi_helper import open_scpi_device, setup_scpi_device, scpi_bool

## Helper functions

In [34]:
def confirm(text, proceed_on_empty=True):
    """
    Ask for confirmation before proceeding to the next step.

    
    """
    while True:
        inp_result = input(text + " (y: proceed, a: abort)").strip()
        if inp_result == 'y' or (proceed_on_empty and inp_result == ''):
            break
        if inp_result == 'a':
            raise KeyboardInterrupt("Aborted")
    
    return True

In [35]:
def input_float(text):
    while True:
        try:
            inp_result = input(text + " (a: abort)").strip()
            if inp_result == 'a':
                raise KeyboardInterrupt("Aborted")
            return float(inp_result)
        except ValueError:
            pass

In [36]:
def current_zero(output_on=False):
    load.write("OUTP OFF")
    assert not scpi_bool(load.query("OUTP?"))
    load.write("CHAN 1")
    assert load.query("CHAN?") == 1
    load.write("MODE:CURR")
    assert load.query("MODE?") == "CURR"
    load.write("CURR 0")
    assert np.isclose(load.query("CURR?"), 0)
    if output_on:
        load.write("OUTP ON")
        assert scpi_bool(load.query("OUTP?"))

### Calibration functions

In [37]:
def current_calibration(load, cal_set):
    curr_range = cal_set["current_range"]
    curr_offset = cal_set["current_offset"]
    flag = cal_set["flag"]
    modifier = cal_set["modifier"]

    confirm(f"CURRENT CALIBRATION for current range {curr_range}A: Proceed?")
    confirm(f"Connect power supply with settings {cal_set['power_supply']['voltage']}V, {cal_set['power_supply']['current']}A")
    
    load.write("CHAN 1")
    assert load.query("CHAN?") == 1
    load.write("MODE:CURR")
    assert load.query("MODE?") == "CURR"
    load.write(f"CURR:RANG {curr_range}")
    assert np.isclose(float(load.query("CURR:RANG?")), curr_range, rtol=1e-4, atol=0.001)

    # Calibrate Current low point
    curr_set = 0.05 * curr_range
    load.write(f"CURR {curr_set}")
    assert np.isclose(float(load.query("CURR?")), curr_set, rtol=1e-4, atol=0.001)
    sleep(5)
    meas_curr_lo = input_float(f"Enter current for low point {curr_set} (A)")
    print(f"Current for low point {curr_set} (A): {meas_curr_lo}")
    assert np.isclose(meas_curr_lo, curr_set, rtol=0.1, atol=0.05)
    load.write(f"CAL:LEV:LOW {meas_curr_lo}")

    # Calibrate Current high point
    curr_set = 0.85 * curr_range * modifier
    load.write(f"CURR {curr_set}")
    assert np.isclose(float(load.query("CURR?")), curr_set, rtol=1e-4, atol=0.001)
    if flag:
        sleep(25)
    meas_curr_hi = input_float(f"Enter current for high point {curr_set} (A)")
    print(f"Current for high point {curr_set} (A): {meas_curr_hi}")
    assert np.isclose(meas_curr_hi, curr_set, rtol=0.1, atol=0.05)
    load.write(f"CAL:LEV:HIGH {meas_curr_hi}")
    if flag:
        load.write(f"CAL:MEAS:HIGH {meas_curr_hi}")
        sleep(25)
    
    # offset compensation or something like that
    if flag:
        curr_set = 4 * curr_range / 3750
        load.write(f"CURR {curr_set}")
        assert np.isclose(float(load.query("CURR?")), curr_set, rtol=1e-4, atol=0.001)
        sleep(25)
        meas_curr_lo = input_float(f"Enter current for low point {curr_set} (A)")
        print(f"Current for low point {curr_set} (A): {meas_curr_lo}")
        assert np.isclose(meas_curr_lo, curr_set, rtol=0.1, atol=0.05)
        load.write(f"CAL:LEV:HIGH {meas_curr_lo - curr_offset}")
        load.write(f"CAL:MEAS:HIGH {meas_curr_lo}")
    else:
        curr_set = 10 * curr_range / 3750
        load.write(f"CURR {curr_set}")
        assert np.isclose(float(load.query("CURR?")), curr_set, rtol=1e-4, atol=0.001)
        meas_curr_lo = input_float(f"Enter current for low point {curr_set} (A)")
        print(f"Current for low point {curr_set} (A): {meas_curr_lo}")
        assert np.isclose(meas_curr_lo, curr_set, rtol=0.1, atol=0.05)
        load.write(f"CAL:LEV:LOW {meas_curr_lo - curr_offset}")

    confirm("Test unit to verify that program and readback values are in spec. Proceed?")
    
    # Calibrate transient
    curr_set = 0.05 * curr_range
    load.write(f"CURR {curr_set}")
    assert np.isclose(float(load.query("CURR?")), curr_set, rtol=1e-4, atol=0.001)
    load.write("TRAN:STAT ON;MODE TOGG;:TRIG:SOUR BUS")
    curr_tran_set = 0.85 * curr_range * modifier
    load.write(f"CURR:TLEV {curr_tran_set}")
    assert np.isclose(float(load.query("CURR:TLEV?")), curr_tran_set, rtol=1e-4, atol=0.001)
    load.write("*TRG")
    if flag:
        sleep(30)
    meas_trig_curr = input_float(f"Enter current for transient high point {curr_tran_set} (A)")
    print(f"Current for transient high point {curr_tran_set} (A): {meas_trig_curr}")
    assert np.isclose(meas_trig_curr, curr_tran_set, rtol=0.1, atol=0.05)
    load.write(f"CAL:TLEV {meas_trig_curr}")
    load.write("TRAN OFF")

    confirm("Test unit to verify that program and readback values are in spec. Proceed?")

SyntaxError: invalid syntax (1335105121.py, line 8)

In [ ]:
def voltage_calibration(load, cal_set):
    volt_hi = cal_set["voltage_high"]
    volt_lo = cal_set["voltage_low"]

    confirm(f"VOLTAGE-CALIBRATION: Proceed?")        
    confirm(f"Connect power supply with settings {cal_set['power_supply']['voltage']}V, {cal_set['power_supply']['current']}A")

    load.write("CHAN 1")
    assert load.query("CHAN?") == 1
    load.write("MODE:VOLT")
    assert load.query("MODE?") == "VOLT"

    # Calibration 5% volt low point
    volt_set = 0.05 * volt_hi
    load.write(f"VOLT {volt_set}")
    assert np.isclose(float(load.query("VOLT?")), volt_set, rtol=1e-4, atol=0.001)
    sleep(5)
    meas_volt_lo = input_float(f"Enter voltage across input terminals for low point {volt_set} (V)")
    print(f"voltage across input terminals for low point {volt_set} (V): {meas_volt_lo}")
    load.write(f"CAL:LEV:LOW {meas_volt_lo}")
    load.write(f"CAL:MEAS:LOW {meas_volt_lo}")
    
    # Calibration 85% volt high point
    volt_set = 0.85 * volt_hi
    load.write(f"VOLT {volt_set}")
    assert np.isclose(float(load.query("VOLT?")), volt_set, rtol=1e-4, atol=0.001)
    sleep(5)
    meas_volt_hi = input_float(f"Enter voltage across input terminals for high point {volt_set} (V)")
    print(f"voltage across input terminals for high point {volt_set} (V): {meas_volt_hi}")
    load.write(f"CAL:LEV:HIGH {meas_volt_hi}")
    load.write(f"CAL:MEAS:HIGH {meas_volt_hi}")

    # Calibration volt low point
    volt_set = volt_lo
    load.write(f"VOLT {volt_set}")
    assert np.isclose(float(load.query("VOLT?")), volt_set, rtol=1e-4, atol=0.001)
    sleep(5)
    meas_volt_lo = input_float(f"Enter voltage across input terminals for low point {volt_set} (V)")
    print(f"voltage across input terminals for low point {volt_set} (V): {meas_volt_lo}")
    load.write(f"CAL:LEV:LOW {meas_volt_lo}")
    load.write(f"CAL:MEAS:LOW {meas_volt_lo}")
    
    # Calibration volt high point
    volt_set = volt_hi
    load.write(f"VOLT {volt_set}")
    assert np.isclose(float(load.query("VOLT?")), volt_set, rtol=1e-4, atol=0.001)
    sleep(5)
    meas_volt_hi = input_float(f"Enter voltage across input terminals for high point {volt_set} (V)")
    print(f"voltage across input terminals for high point {volt_set} (V): {meas_volt_hi}")
    load.write(f"CAL:LEV:HIGH {meas_volt_hi}")
    load.write(f"CAL:MEAS:HIGH {meas_volt_hi}")
    
    # Calibrate transient level
    input("Test unit to verify that program and readback values are in spec. Proceed?")
    volt_set = volt_lo
    load.write(f"VOLT {volt_set}")
    assert np.isclose(float(load.query("VOLT?")), volt_set, rtol=1e-4, atol=0.001)
    load.write("TRAN:STAT ON;MODE TOGG;:TRIG:SOUR BUS")
    volt_tran_set = volt_hi
    load.write(f"VOLT:TLEV {volt_tran_set}")
    assert np.isclose(float(load.query("VOLT:TLEV?")), volt_tran_set, rtol=1e-4, atol=0.001)
    load.write("*TRG")
    meas_tran_volt = input_float(f"Enter voltage for transient high point {volt_tran_set} (V)")
    print(f"voltage across input terminals for transient high point {volt_tran_set} (V): {meas_tran_volt}")
    assert np.isclose(meas_tran_volt, volt_tran_set, rtol=0.1, atol=0.05)
    load.write(f"CAL:TLEV {meas_tran_volt}")
    load.write("TRAN OFF")
    
    confirm("Test unit to verify that program and readback values are in spec. Proceed?")

In [ ]:
def resistance_calibration(load, cal_set):
    res_range = cal_set["res_range"]
    res_hi = cal_set["res_high"]
    res_lo = cal_set["res_low"] * cal_set["modifier"]
    flag = cal_set["flag"]

    confirm(f"RESISTANCE CALIBRATION for resistance range {res_range} Ohm. Proceed?")
    confirm(f"Connect power supply with settings {cal_set['power_supply']['voltage']}V, {cal_set['power_supply']['current']}A")

    load.write("CHAN 1")
    assert load.query("CHAN?") == 1
    load.write("MODE:RES")
    assert load.query("MODE?") == "RES"
    load.write(f"RES:RANG {res_range}")

    # Calibration res high point
    res_set = res_hi
    load.write(f"RES {res_set}")
    assert np.isclose(float(load.query("RES?")), res_set, rtol=1e-2, atol=0.001)
    sleep(5)
    meas_volt_hi = input_float(f"Enter voltage across input terminals for high point {res_set} Ohm (V)")
    print(f"voltage across input terminals for high point {res_set} Ohm (V): {meas_volt_hi}")
    meas_curr_hi = input_float(f"Enter current for high point {res_set} Ohm (A)")
    print(f"current for high point {res_set} Ohm (A): {meas_curr_hi}")
    calc_res_hi = meas_volt_hi / meas_curr_hi
    load.write(f"CAL:LEV:HIGH {calc_res_hi}")
        
    # Calibration res low point
    res_set = res_lo
    load.write(f"RES {res_set}")
    assert np.isclose(float(load.query("RES?")), res_set, rtol=1e-2, atol=0.001)
    sleep(5)
    meas_volt_lo = input_float(f"Enter voltage across input terminals for low point {res_set} Ohm (V)")
    print(f"voltage across input terminals for low point {res_set} Ohm (V): {meas_volt_lo}")
    meas_curr_lo = input_float(f"Enter current for low point {res_set} Ohm (A)")
    print(f"current for high point {res_set} Ohm (A): {meas_curr_lo}")
    cal_res_lo = meas_volt_lo / meas_curr_lo
    load.write(f"CAL:LEV:LOW {cal_res_lo}")

    confirm("Test unit to verify that program and readback values are in spec. Proceed?")

    # Calibrate transient level
    res_set = res_hi if flag else res_lo
    load.write(f"RES {res_set}")
    assert np.isclose(float(load.query("RES?")), res_set, rtol=1e-2, atol=0.001)
    load.write("TRAN:STAT ON;MODE TOGG;:TRIG:SOUR BUS")
    res_tran_set = res_lo if flag else res_hi
    load.write(f"RES:TLEV {res_tran_set}")
    assert np.isclose(float(load.query("RES:TLEV?")), res_tran_set, rtol=1e-2, atol=0.001)
    load.write("*TRG")

    meas_tran_volt = input_float(f"Enter voltage for transient high point {res_tran_set} (V)")
    print(f"voltage across input terminals for transient high point {res_tran_set} Ohm (V): {meas_tran_volt}")
    meas_tran_curr = input_float(f"Enter current for transient high point {res_tran_set} (A)")
    print(f"current for transient high point {res_tran_set} Ohm (A): {meas_tran_curr}")
    calc_tran_res = meas_tran_volt / meas_tran_curr
    assert np.isclose(calc_tran_res, res_tran_set, rtol=0.1, atol=0.05)
    load.write(f"CAL:TLEV {calc_tran_res}")
    load.write("TRAN OFF")
    
    confirm("Test unit to verify that program and readback values are in spec. Proceed?")

## Perform adjustment

In [ ]:
rm = pyvisa.ResourceManager()
res = rm.list_resources()
res

In [ ]:
load = open_scpi_device(rm, load_res_id, "HEWLETT-PACKARD,6060B")
setup_scpi_device(load)

In [ ]:
# Prepare calibration
load.write("CAL OFF")
load.write("CAL ON")
current_zero(output_on=True)

In [ ]:
# Run through all calibration steps
current_calibration(load, cal_set_curr_rng_hi, 1)
current_calibration(load, cal_set_curr_rng_lo, 1)
voltage_calibration(load, cal_set_volt)
resistance_calibration(load, cal_set_res_rng_lo, 1)
resistance_calibration(load, cal_set_res_rng_med, 1)
resistance_calibration(load, cal_set_res_rng_hi, 2)

In [ ]:
# Finalize calibration
if cal_save:
    load.write("CAL:SAVE")
    load.write("CAL OFF")

In [ ]:
current_zero()
confirm("Switch unit off. Some internal values are not re-calculated until the unit was power cycled. Ok")